___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />



# <font color= paletvioletred > Laboratorio 4 - Analisis Fundamental</font>




### Integrantes: 
- <Strong> Chavez Jacobo Diana Teresa </Strong> 


- <Strong> Calderón Erape Jazmín </Strong>

#### 14 Abril 2019

___
### <Strong> <font color = skyblue> I. Motivación del Laboratorio </Strong>

___
### <Strong> <font color = skyblue> II. Objetivos específicos </Strong>

In [1]:
# -- ------------------ Importar modulos a utilizar ------------------ -- #

import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
#import plotly.graph_objs as go
#import plotly as py
from datetime import datetime, date, time, timedelta
import calendar

In [2]:
# -- ------------------ Parametros para Data Frame ------------------ -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [3]:
# -- ------------------ Parametros para OANDA ------------------ -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

A1_OA_Ak =  "6ff" + "6703d22fd972709992df543492afe" + "-f954a9914f9b39f72fba177a23eb243" + "b" #mi api


In [4]:
# -- ------------------ Estructuras de datos ------------------ -- #



# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, segun fechas y granularidad
# -- -- Fecha, Apertura, Maximo, Minimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrio operacion: compra/venta",
# -- -- "Se cerro operacion: Con perdia de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion maximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operacion: 1) TP o SL, 2) Regla con indicador

indice = [1, 2, 3, 4, 5]
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s

In [5]:
# -- ------------------ Inicializar API de OANDA ------------------ -- #

api = API(access_token=A1_OA_Ak)

In [6]:
# -- ------------------ Función de fechas para descargar precios ------------------ -- #

from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

In [7]:
# -- ------------------ vector de fechas por periodos de 15 dias ------------------ -- #

start_date = date(2017, 4, 14)
end_date = date(2019, 4, 10)
fechas = date_range(start_date, end_date, 15, 'days')

F1 = fechas[0].strftime('%Y-%m-%dT%H:%M:%S')
F2 = fechas[1].strftime('%Y-%m-%dT%H:%M:%S')

___
### <Strong> <font color = skyblue> DataFrame1 Precios peso-dólar </Strong>

In [8]:
params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

In [9]:
lis = []
for i in range(len(A1_Hist['candles'])-1):
        lis.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lis)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

In [10]:
# Muestra de precios históricos del peso dólar.

pd_hist

,TimeStamp,Open,High,Low,Close
0,2017-04-14 04:00:00,18.57634,18.57638,18.57450,18.57556
1,2017-04-14 04:05:00,18.57564,18.57619,18.57479,18.57558
2,2017-04-14 04:10:00,18.57560,18.57568,18.57540,18.57547
3,2017-04-14 04:15:00,18.57550,18.57550,18.57534,18.57538
4,2017-04-14 04:20:00,18.57536,18.57560,18.56932,18.57492
...,...,...,...,...,...
3070,2017-04-28 20:30:00,18.82808,18.82991,18.82721,18.82988
3071,2017-04-28 20:35:00,18.82985,18.83002,18.82970,18.82970
3072,2017-04-28 20:40:00,18.82972,18.82973,18.82614,18.82672
3073,2017-04-28 20:45:00,18.82666,18.82719,18.82492,18.82542


___
### <Strong> <font color = skyblue> Indicador: Consumer Price Index ex Food & Energy (MoM)</Strong>

In [11]:
indicador = pd.read_csv('https://raw.githubusercontent.com/Dianachav/bebe/master/Consumer%20Price%20Index%20ex%20Food%20%26%20Energy%20(MoM).csv')
pd.set_option('display.max_rows', 25)
indicador

,DateTime,Actual,Consensus,Previous
0,4/10/19 12:30,0.1,0.2,0.1
1,3/12/19 12:30,0.1,0.2,0.2
2,2/13/19 13:30,0.2,0.2,0.2
3,1/11/19 13:30,0.2,0.2,0.2
4,12/12/18 13:30,0.2,0.2,0.2
5,11/14/18 13:30,0.2,0.2,0.1
6,10/11/18 12:30,0.1,0.2,0.1
7,9/13/18 12:30,0.1,0.2,0.2
8,8/10/18 12:30,0.2,0.2,0.2
9,7/12/18 12:30,0.2,0.2,0.2


___
### <Strong> <font color = skyblue> Escenarios </Strong>

In [12]:
escenarios = []

for i in range(len(indicador['Actual'])):
    if indicador['Actual'][i]>= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario A
        escenarios.append('A')
    elif indicador['Actual'][i]>= indicador['Consensus'][i]<=indicador['Previous'][i]:#Escenario B
        escenarios.append('B')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario C
        escenarios.append('C')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]<=indicador['Previous'][i]: #Escenario D
        escenarios.append('D')

In [13]:
tabla=pd.DataFrame(columns=['DateTime','Actual','Consensus', 'Previous', 'Scene'])
tabla['DateTime'] = indicador['DateTime']
tabla['Actual'] = indicador['Actual']
tabla['Consensus'] = indicador['Consensus']
tabla['Previous'] = indicador['Previous']
tabla['Scene'] = escenarios
pd.set_option('display.max_rows', 25)
tabla.set_index('DateTime')


,Actual,Consensus,Previous,Scene
DateTime,,,,
4/10/19 12:30,0.1,0.2,0.1,C
3/12/19 12:30,0.1,0.2,0.2,C
2/13/19 13:30,0.2,0.2,0.2,A
1/11/19 13:30,0.2,0.2,0.2,A
12/12/18 13:30,0.2,0.2,0.2,A
11/14/18 13:30,0.2,0.2,0.1,A
10/11/18 12:30,0.1,0.2,0.1,C
9/13/18 12:30,0.1,0.2,0.2,C
8/10/18 12:30,0.2,0.2,0.2,A


___
### <Strong> <font color = skyblue> Ventanas </Strong>

In [14]:
def ventana(F1,F2):
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    lista = []

    for i in range(len(A1_Hist['candles'])-1):
            lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                          'Open': A1_Hist['candles'][i]['mid']['o'],
                          'High': A1_Hist['candles'][i]['mid']['h'],
                          'Low': A1_Hist['candles'][i]['mid']['l'],
                          'Close': A1_Hist['candles'][i]['mid']['c']})

    pd_hist = pd.DataFrame(lista)
    pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    return(pd_hist)

___
### <Strong> <font color = skyblue> Características </Strong>

$C1$ = $High_{max}$ - $Low_{min}$  -> Volatilidad

$C2$ = $Open_{t0}$ - $Close_{t10}$  -> Dirección

$C3$ = $Open_{t0}$ - $Low_{t10}$   -> Draw Down

$C4$ = $Open_{t0}$ - $High_{t10}$   -> Draw Up

Donde:

${max}$ = Máximo valor de la ventana de precios

${min}$ = Mínimo valor de la ventana de precios

$t_{0}$ = Tiempo de ocurrencia del indicador

$t_{10}$ = Último dato de la ventana de precios

In [15]:
def caract(v):
    #v = ventana(F1 = "2019-04-10T11:40:00Z",F2 = "2019-04-10T13:30:00Z")
    o = v['Open'].astype(float)
    c = v['Close'].astype(float)
    l = v['Low'].astype(float)
    h = v['High'].astype(float)

    caract1= max(h)-min(l)
    caract2= o[10]-c[20]
    caract3= o[10]-l[20]
    caract4= o[10]-h[20]
    c1=round(caract1,3)
    c2=round(caract2,3)
    c3=round(caract3,3)
    c4=round(caract4,3)
    return(c1,c2,c3,c3)

In [16]:
# -- --------------------------------------Ventana indicador 1 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #
v = ventana(F1 = "2019-04-10T11:40:00Z",F2 = "2019-04-10T13:30:00Z")
v1=caract(v)

In [17]:
# -- --------------------------------------Ventana indicador 2 ---------------------------------- #
# -- ------------------------------------------------------------- --------------------------- -- #

v = ventana(F1 = "2019-03-12T11:40:00Z",F2 = "2019-03-12T13:30:00Z")
v2=caract(v)

In [18]:
# -- --------------------------------------Ventana indicador 3 ---------------------------------- #

v = ventana(F1 = "2019-02-13T12:40:00Z",F2 = "2019-02-13T14:30:00Z")
v3=caract(v)

In [19]:
# -- --------------------------------------Ventana indicador 4 ---------------------------------- #

v = ventana(F1 = "2019-01-11T12:40:00Z",F2 = "2019-01-11T14:30:00Z")
v4=caract(v)

In [20]:
# -- --------------------------------------Ventana indicador 5 ---------------------------------- #

v = ventana(F1 = "2018-12-12T12:40:00Z",F2 = "2018-12-12T14:30:00Z")
v5=caract(v)

In [21]:
# -- --------------------------------------Ventana indicador 6 ---------------------------------- #

v = ventana(F1 = "2018-11-14T12:40:00Z",F2 = "2018-11-14T14:30:00Z")
v6=caract(v)

In [22]:
# -- --------------------------------------Ventana indicador 7 ---------------------------------- #

v = ventana(F1 = "2018-10-11T11:40:00Z",F2 = "2018-10-11T13:30:00Z")
v7=caract(v)

In [23]:
# -- --------------------------------------Ventana indicador 8 ---------------------------------- #

v = ventana(F1 = "2018-09-13T11:40:00Z",F2 = "2018-09-13T13:30:00Z")
v8=caract(v)

In [24]:
# -- --------------------------------------Ventana indicador 9 ---------------------------------- #

v = ventana(F1 = "2018-08-10T11:40:00Z",F2 = "2018-08-10T13:30:00Z")
v9=caract(v)

In [25]:
# -- --------------------------------------Ventana indicador 10 ---------------------------------- #

v = ventana(F1 = "2018-07-12T11:40:00Z",F2 = "2018-07-12T13:30:00Z")
v10=caract(v)


In [26]:
# -- --------------------------------------Ventana indicador 11 ---------------------------------- #

v = ventana(F1 = "2018-06-12T11:40:00Z",F2 = "2018-06-12T13:30:00Z")
v11=caract(v)

In [27]:
# -- --------------------------------------Ventana indicador 12 ---------------------------------- #

v = ventana(F1 = "2018-05-10T11:40:00Z",F2 = "2018-05-10T13:30:00Z")
v12=caract(v)

In [28]:
# -- --------------------------------------Ventana indicador 13 ---------------------------------- #

v = ventana(F1 = "2018-04-11T11:40:00Z",F2 = "2018-04-11T13:30:00Z")
v13=caract(v)

In [29]:
# -- --------------------------------------Ventana indicador 14 ---------------------------------- #

v = ventana(F1 = "2018-03-13T11:40:00Z",F2 = "2018-03-13T13:30:00Z")
v14=caract(v)

In [30]:
# -- --------------------------------------Ventana indicador 15 ---------------------------------- #

v = ventana(F1 = "2018-02-14T12:40:00Z",F2 = "2018-02-14T14:30:00Z")
v15=caract(v)


In [31]:
# -- --------------------------------------Ventana indicador 16 ---------------------------------- #

v = ventana(F1 = "2018-01-12T12:40:00Z",F2 = "2018-01-12T14:30:00Z")
v16=caract(v)

In [32]:
# -- --------------------------------------Ventana indicador 17 ---------------------------------- #

v = ventana(F1 = "2017-12-13T12:40:00Z",F2 = "2017-12-13T14:30:00Z")
v17=caract(v)

In [33]:
# -- --------------------------------------Ventana indicador 18 ---------------------------------- #

v = ventana(F1 = "2017-11-15T11:40:00Z",F2 = "2017-11-15T13:30:00Z")
v18=caract(v)

In [34]:
# -- --------------------------------------Ventana indicador 19 ---------------------------------- #

v = ventana(F1 = "2017-10-13T11:40:00Z",F2 = "2017-10-13T13:30:00Z")
v19=caract(v)

In [35]:
# -- --------------------------------------Ventana indicador 20 ---------------------------------- #

v = ventana(F1 = "2017-09-14T11:40:00Z",F2 = "2017-09-14T13:30:00Z")
v20=caract(v)

In [36]:
# -- --------------------------------------Ventana indicador 21 ---------------------------------- #

v = ventana(F1 = "2017-08-11T11:40:00Z",F2 = "2017-08-11T13:30:00Z")
v21=caract(v)

In [37]:
# -- --------------------------------------Ventana indicador 22 ---------------------------------- #

v = ventana(F1 = "2017-07-14T11:40:00Z",F2 = "2017-07-14T13:30:00Z")
v22=caract(v)

In [38]:
# -- --------------------------------------Ventana indicador 23 ---------------------------------- #

v = ventana(F1 = "2017-06-14T11:40:00Z",F2 = "2017-06-14T13:30:00Z")
v23=caract(v)


In [39]:
# -- --------------------------------------Ventana indicador 24 ---------------------------------- #

v = ventana(F1 = "2017-05-12T11:40:00Z",F2 = "2017-05-12T13:30:00Z")
v24=caract(v)

In [40]:
# -- --------------------------------------Ventana indicador 25 ---------------------------------- #

v = ventana(F1 = "2017-04-14T11:40:00Z",F2 = "2017-04-14T13:30:00Z")
v25=caract(v)

___
### <Strong> <font color = skyblue> DataFrame5 Escenario-Caracteristicas </Strong>

In [42]:
indice = tabla['Scene']
columnas = ['Volatilidad','Dirección', 'Draw Down', 'Draw Up']
df = pd.DataFrame(index=indice, columns=columnas)
df.iloc[0] = v1
df.iloc[1] = v2
df.iloc[2] = v3
df.iloc[3] = v4
df.iloc[4] = v5
df.iloc[5] = v6
df.iloc[6] = v7
df.iloc[7] = v8
df.iloc[8] = v9
df.iloc[9] = v10
df.iloc[10] = v11
df.iloc[11] = v12
df.iloc[12] = v13
df.iloc[13] = v14
df.iloc[14] = v15
df.iloc[15] = v16
df.iloc[16] = v17
df.iloc[17] = v18
df.iloc[18] = v19
df.iloc[19] = v20
df.iloc[20] = v11
df.iloc[21] = v22
df.iloc[22] = v23
df.iloc[23] = v24
df.iloc[24] = v25
df

,Volatilidad,Dirección,Draw Down,Draw Up
Scene,,,,
C,0.06,0.022,0.025,0.025
C,0.141,0.063,0.074,0.074
A,0.102,-0.046,-0.044,-0.044
A,0.089,-0.037,-0.018,-0.018
A,0.119,0.035,0.052,0.052
A,0.128,0.037,0.054,0.054
C,0.141,0.04,0.042,0.042
C,0.151,0.122,0.136,0.136
A,0.182,-0.149,-0.107,-0.107
